# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 11:24AM,252824,21,635896,"NBTY Global, Inc.",Released
1,Dec 12 2022 11:22AM,252823,21,635894,"NBTY Global, Inc.",Released
2,Dec 12 2022 11:12AM,252822,19,MSP216657,"Methapharm, Inc.",Released
3,Dec 12 2022 11:12AM,252822,19,MSP216660,"Methapharm, Inc.",Released
4,Dec 12 2022 11:10AM,252821,19,8893136,VITABIOTICS USA INC.,Released
5,Dec 12 2022 10:39AM,252819,10,0086237796,ISDIN Corporation,Released
6,Dec 12 2022 10:39AM,252819,10,0086237797,ISDIN Corporation,Released
7,Dec 12 2022 10:39AM,252819,10,0086237803,ISDIN Corporation,Released
8,Dec 12 2022 10:39AM,252819,10,0086237808,ISDIN Corporation,Released
9,Dec 12 2022 10:39AM,252819,10,0086237822,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,252819,Released,24
20,252821,Released,1
21,252822,Released,2
22,252823,Released,1
23,252824,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252819,NaN,24.0
252821,NaN,1.0
252822,NaN,2.0
252823,NaN,1.0
252824,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252777,0.0,1.0
252778,0.0,1.0
252779,1.0,0.0
252783,1.0,0.0
252784,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252777,0,1
252778,0,1
252779,1,0
252783,1,0
252784,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252777,0,1
1,252778,0,1
2,252779,1,0
3,252783,1,0
4,252784,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252777,,1
1,252778,,1
2,252779,1,
3,252783,1,
4,252784,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 11:24AM,252824,21,"NBTY Global, Inc."
1,Dec 12 2022 11:22AM,252823,21,"NBTY Global, Inc."
2,Dec 12 2022 11:12AM,252822,19,"Methapharm, Inc."
4,Dec 12 2022 11:10AM,252821,19,VITABIOTICS USA INC.
5,Dec 12 2022 10:39AM,252819,10,ISDIN Corporation
29,Dec 12 2022 10:35AM,252818,10,ISDIN Corporation
32,Dec 12 2022 10:30AM,252816,19,"Granules Pharmaceuticals, Inc."
33,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC"
34,Dec 12 2022 10:28AM,252813,10,Methapharm-G
35,Dec 12 2022 10:26AM,252812,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 11:24AM,252824,21,"NBTY Global, Inc.",,1
1,Dec 12 2022 11:22AM,252823,21,"NBTY Global, Inc.",,1
2,Dec 12 2022 11:12AM,252822,19,"Methapharm, Inc.",,2
3,Dec 12 2022 11:10AM,252821,19,VITABIOTICS USA INC.,,1
4,Dec 12 2022 10:39AM,252819,10,ISDIN Corporation,,24
5,Dec 12 2022 10:35AM,252818,10,ISDIN Corporation,,3
6,Dec 12 2022 10:30AM,252816,19,"Granules Pharmaceuticals, Inc.",,1
7,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",,1
8,Dec 12 2022 10:28AM,252813,10,Methapharm-G,,1
9,Dec 12 2022 10:26AM,252812,10,Bio-PRF,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 11:24AM,252824,21,"NBTY Global, Inc.",1,
1,Dec 12 2022 11:22AM,252823,21,"NBTY Global, Inc.",1,
2,Dec 12 2022 11:12AM,252822,19,"Methapharm, Inc.",2,
3,Dec 12 2022 11:10AM,252821,19,VITABIOTICS USA INC.,1,
4,Dec 12 2022 10:39AM,252819,10,ISDIN Corporation,24,
5,Dec 12 2022 10:35AM,252818,10,ISDIN Corporation,3,
6,Dec 12 2022 10:30AM,252816,19,"Granules Pharmaceuticals, Inc.",1,
7,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",1,
8,Dec 12 2022 10:28AM,252813,10,Methapharm-G,1,
9,Dec 12 2022 10:26AM,252812,10,Bio-PRF,18,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 11:24AM,252824,21,"NBTY Global, Inc.",1,
1,Dec 12 2022 11:22AM,252823,21,"NBTY Global, Inc.",1,
2,Dec 12 2022 11:12AM,252822,19,"Methapharm, Inc.",2,
3,Dec 12 2022 11:10AM,252821,19,VITABIOTICS USA INC.,1,
4,Dec 12 2022 10:39AM,252819,10,ISDIN Corporation,24,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 11:24AM,252824,21,"NBTY Global, Inc.",1.0,NaN
1,Dec 12 2022 11:22AM,252823,21,"NBTY Global, Inc.",1.0,NaN
2,Dec 12 2022 11:12AM,252822,19,"Methapharm, Inc.",2.0,NaN
3,Dec 12 2022 11:10AM,252821,19,VITABIOTICS USA INC.,1.0,NaN
4,Dec 12 2022 10:39AM,252819,10,ISDIN Corporation,24.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 11:24AM,252824,21,"NBTY Global, Inc.",1.0,0.0
1,Dec 12 2022 11:22AM,252823,21,"NBTY Global, Inc.",1.0,0.0
2,Dec 12 2022 11:12AM,252822,19,"Methapharm, Inc.",2.0,0.0
3,Dec 12 2022 11:10AM,252821,19,VITABIOTICS USA INC.,1.0,0.0
4,Dec 12 2022 10:39AM,252819,10,ISDIN Corporation,24.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2780858,162.0,0.0
12,505572,1.0,1.0
19,1264058,6.0,0.0
20,758341,3.0,0.0
21,505647,2.0,0.0
22,252779,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2780858,162.0,0.0
1,12,505572,1.0,1.0
2,19,1264058,6.0,0.0
3,20,758341,3.0,0.0
4,21,505647,2.0,0.0
5,22,252779,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,162.0,0.0
1,12,1.0,1.0
2,19,6.0,0.0
3,20,3.0,0.0
4,21,2.0,0.0
5,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,162.0
1,12,Released,1.0
2,19,Released,6.0
3,20,Released,3.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,21,22
Status,,,,,,
Executing,0.0,1.0,0.0,0.0,0.0,1.0
Released,162.0,1.0,6.0,3.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,21,22
0,Executing,0.0,1.0,0.0,0.0,0.0,1.0
1,Released,162.0,1.0,6.0,3.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,21,22
0,Executing,0.0,1.0,0.0,0.0,0.0,1.0
1,Released,162.0,1.0,6.0,3.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()